# LangChain 中的Agents如何实现？
- 在agent与tool之间共享记忆，让任务更精准
    - LLM
    - Tools
    - Memory

# 在agent与tool之间共享记忆
- 自定义一个工具用来LLMChain来总结内容
- 使用readonlymemory来共享记忆
- 观察共享与不共享的区别

In [1]:
import os
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv()

True

In [13]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain_openai import OpenAI, ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate,MessagesPlaceholder
from langchain.utilities import SerpAPIWrapper

大模型

In [14]:
llm = ChatOpenAI(
    temperature = 0,
    model="gpt-3.5-turbo",
)

创建一条链来总结对话

In [15]:
template = """以下是一段AI机器人和人类的对话:
{chat_history}
根据输入和上面的对话记录写一份对话总结.
输入: {input}"""

prompt = PromptTemplate(
    input_variables=["input","chat_history"],
    template=template,
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)
readonlymemory = ReadOnlySharedMemory(memory=memory)
summary_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,
)

构建工具

In [16]:
#搜索工具
search = SerpAPIWrapper()
#总结工具
def SummaryChainFun(history):
    print("\n==============总结链开始运行==============")
    print("输入历史: ",history)
    summary_chain.run(history)

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="当需要了解实时的信息或者你不知道的事时候可以使用搜索工具",
    ),
    Tool(
        name="Summary",
        func=SummaryChainFun,
        description="当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用",
    ),
]
print(tools)

[Tool(name='Search', description='当需要了解实时的信息或者你不知道的事时候可以使用搜索工具', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='4ba6f3d3a1010b2252ca53ebe8fc77c1727eadbc894149a03a5fb5039d86194c', aiosession=None)>), Tool(name='Summary', description='当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用', func=<function SummaryChainFun at 0x000002800B5AEFC0>)]


In [17]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

In [18]:
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    memory=memory,
)

In [19]:
print(agent_chain.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: 当需要了解实时的信息或者你不知道的事时候可以使用搜索工具
Summary: 当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [20]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"
{chat_history}
Question: {input}
{agent_scratchpad}"""

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={
        "prefix":prefix,
        "suffix":suffix,
        "agent_scratchpad":MessagesPlaceholder("agent_scratchpad"),
        "chat_history":MessagesPlaceholder("chat_history"),
        "input":MessagesPlaceholder("input"),
    },
    memory=memory,
)
print(agent_chain.agent.llm_chain.prompt.template)

Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:

Search: 当需要了解实时的信息或者你不知道的事时候可以使用搜索工具
Summary: 当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"
{chat_history}
Question: {input}
{agent_scratchpad}


In [22]:
agent_chain.run(input="美国第45任总统是谁?,中文输出")



> Entering new AgentExecutor chain...
Thought: I need to find out who the 45th president of the United States is.
Action: Search
Action Input: 美国第45任总统是谁?
Observation: ['州内建县、镇、市和乡各级，各级均设有选任政府。 美国的政府权力和职能由三个政府分支机构担当：立法机关、司法机关和行政机关。《宪法》第1条 ...', '1789年，选举人团投票一致通过，选举出乔治·华盛顿为第1任美国总统。第7任总统安德鲁·杰克逊，在1828年多数限制非土地所有者投票权法律被废除后，成为首位由全体白人男性 ...', '唐纳德·特朗普（Donald Trump，1946年6月14日—），出生于美国纽约，祖籍德国巴伐利亚自由州，德裔美国共和党籍政治家、企业家、房地产商人、电视人，第45任美国 ...', '在1951年美国宪法第二十二修正案通过之后，总统仅限于2个当选任期，总共不超过10年。若在任总统死亡、辞职、被罢免、被弹劾，则由副总统接替其职务。总统必须年满35岁，在 ...', '赢得11月8日总统大选的特朗普在拥有538个成员的选举人团票选中获得所需的270张选举人票，正式成为美国第45任总统。', '美国第45任总统唐纳德·特朗普在美国政治史上可谓独树一帜，其权力核心圈也开了不少先河，从年龄、财富到人生哲学，为美国历史和白宫历史添了不少"第 ...', '在内政上，特朗普在就任当天便以行政命令的方式叫停“奥巴马医保”，此后又动用行政权力削减其实际效力。在移民问题上，特朗普相继出台强硬举措，“限移令”、 ...', '共和党总统候选人特朗普在佛罗里达州，俄亥俄州，北卡罗来纳州，爱荷华州和宾夕法尼亚等关键的州展示了自己的实力，赢得的代表数字升到274。 赢得加利福尼亚州和内华达州 ...', '2016年年底的美国总统选举，共和党特朗普胜选，出任美国第45任总统，各国舆论为之轰动。人们关心此次选举所反映的美国社会诸多问题，关心2017年新一届美国政府的政策 ...', '【编者按】当地时间11月9日，美国总统大选最终结果出炉：共和党候选人唐纳德·特朗普击败民主党候选人希拉里·克林顿，成功当选为美国第57届、第

'唐纳德·特朗普'

In [23]:
agent_chain.run(input="他的夫人叫什么名字?")



> Entering new AgentExecutor chain...
Thought: I need to find out the name of the 45th President of the United States' wife.
Action: Search
Action Input: Name of Donald Trump's wife
Observation: [{'title': 'Barron Trump selected to be a Florida delegate at Republican National Convention', 'link': 'https://www.cnn.com/2024/05/09/politics/barron-trump-delegate-republican-national-convention/index.html', 'source': 'CNN', 'date': '24 hours ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSjru2awym3kbb1D5O5HqwqC3l8W_gpWat09X93iFSi6ynpmStMlNbETP0&usqp=CAI&s'}, {'title': "Barron Trump: Donald Trump's youngest son to play role at Republican convention", 'link': 'https://www.bbc.com/news/world-us-canada-68982873', 'source': 'BBC', 'date': '23 hours ago', 'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTvGYV7afdnzbId8Xr79sv-IIqxTfXqFWTHwIuyhByNlUMoFLnlIj5btrs&usqp=CAI&s'}, {'title': 'Takeaways from Day 14 of the Donald Trump hush money trial', 'link

'Melania Trump'

In [24]:
# 记忆内容
print(agent_chain.memory.buffer)

[HumanMessage(content='美国第45任总统是谁?'), AIMessage(content='Donald Trump'), HumanMessage(content='美国第45任总统是谁?,中文输出'), AIMessage(content='唐纳德·特朗普'), HumanMessage(content='他的夫人叫什么名字?'), AIMessage(content='Melania Trump')]


In [25]:
agent_chain.run(input="我们都聊了什么？")



> Entering new AgentExecutor chain...
Thought: I need to summarize the conversation to answer the question.
Action: Summary
Action Input: "美国第45任总统是谁? Donald Trump 美国第45任总统是谁?,中文输出 唐纳德·特朗普 他的夫人叫什么名字? Melania Trump"
==============总结链开始运行==============
输入历史:  美国第45任总统是谁? Donald Trump 美国第45任总统是谁?,中文输出 唐纳德·特朗普 他的夫人叫什么名字? Melania Trump


> Entering new LLMChain chain...
Prompt after formatting:
以下是一段AI机器人和人类的对话:
[]
根据输入和上面的对话记录写一份对话总结.
输入: 美国第45任总统是谁? Donald Trump 美国第45任总统是谁?,中文输出 唐纳德·特朗普 他的夫人叫什么名字? Melania Trump

> Finished chain.

Observation: None
Thought:I need to summarize the conversation to answer the question.
Action: Summary
Action Input: "美国第45任总统是谁? Donald Trump 美国第45任总统是谁?,中文输出 唐纳德·特朗普 他的夫人叫什么名字? Melania Trump"
==============总结链开始运行==============
输入历史:  美国第45任总统是谁? Donald Trump 美国第45任总统是谁?,中文输出 唐纳德·特朗普 他的夫人叫什么名字? Melania Trump


> Entering new LLMChain chain...
Prompt after formatting:
以下是一段AI机器人和人类的对话:
[]
根据输入和上面的对话记录写一份对话总结.
输入: 美国第45任总统是谁? Donald Trump 美国第45任总统是谁?,中文输出 唐纳德·

'The conversation included discussing who the 45th President of the United States was (Donald Trump), his Chinese name (唐纳德·特朗普), and the name of his wife (Melania Trump).'

In [27]:
agent_chain.run(input="帮我总结下目前的对话内容，给我5岁的儿子看看。中文输出")



> Entering new AgentExecutor chain...
Thought: I should use the Summary tool to summarize the conversation in Chinese for a 5-year-old.
Action: Summary
Action Input: "The conversation was about who the President of the United States was, his name in Chinese, and the name of his wife."
==============总结链开始运行==============
输入历史:  The conversation was about who the President of the United States was, his name in Chinese, and the name of his wife.


> Entering new LLMChain chain...
Prompt after formatting:
以下是一段AI机器人和人类的对话:
[]
根据输入和上面的对话记录写一份对话总结.
输入: The conversation was about who the President of the United States was, his name in Chinese, and the name of his wife.

> Finished chain.

Observation: None
Thought:I need to provide a simple summary in Chinese for a 5-year-old.
Final Answer: 这次对话是关于美国总统是谁，他的中文名字，以及他妻子的名字。

> Finished chain.


'这次对话是关于美国总统是谁，他的中文名字，以及他妻子的名字。'

In [28]:
#查看短时记忆
print(agent_chain.memory.buffer)

[HumanMessage(content='美国第45任总统是谁?'), AIMessage(content='Donald Trump'), HumanMessage(content='美国第45任总统是谁?,中文输出'), AIMessage(content='唐纳德·特朗普'), HumanMessage(content='他的夫人叫什么名字?'), AIMessage(content='Melania Trump'), HumanMessage(content='我们都聊了什么？'), AIMessage(content='The conversation included discussing who the 45th President of the United States was (Donald Trump), his Chinese name (唐纳德·特朗普), and the name of his wife (Melania Trump).'), HumanMessage(content='帮我总结下目前的对话内容，给我5岁的儿子看看'), AIMessage(content='The conversation was about who the President of the United States was, his name in Chinese, and the name of his wife.'), HumanMessage(content='帮我总结下目前的对话内容，给我5岁的儿子看看。中文输出'), AIMessage(content='这次对话是关于美国总统是谁，他的中文名字，以及他妻子的名字。')]
